In [1]:
import os
import numpy as np
import torch

from utils.models import Regressor
# from utils.mmfit_data import unfold_acc, train_datasets, val_datasets, SequentialStridedSampler, TRAIN_W_IDS, VAL_W_IDS
import utils.mmfit_data as mmfit
from utils import config
from utils import utils

In [2]:
# >>> CNN <<< #
pose2imu_model = Regressor(
    in_ch=config.in_ch, num_joints=config.num_joints, window_length=config.sensor_window_length
).to(config.device)
best_seed = config.best_pose2imu_seed
name = config.pose2imu_model_name + f"[{best_seed}](model).pth"
latest_model = utils.find_latest_model(name)
print(f"loading: {latest_model}")
pose2imu_model.load_state_dict(torch.load(latest_model, map_location=config.device))

loading: ../train_out/19.02/03.18_pose2imu[2](model).pth


<All keys matched successfully>

In [4]:
# save simulated acc data
for w_id, dataset in zip(
    config.TRAIN_W_IDS + config.VAL_W_IDS + config.TEST_W_IDS,
    mmfit.train_datasets.datasets + mmfit.val_datasets.datasets + mmfit.test_datasets.datasets
):
    real_acc = dataset.acc  # (N, 5)
    real_acc_data = real_acc[:, 2:].permute(1, 0)  # (3, N)
    pred_acc = torch.empty_like(real_acc_data).to(config.device) # (3, N)
    mmfit.unfold(model=pose2imu_model, dataset=dataset, pred_acc=pred_acc)

    # save pred_acc
    sim_acc = torch.empty_like(real_acc)  # (N, 5)
    sim_acc[:, :2] = real_acc[:, :2]
    sim_acc[:, 2:] = pred_acc.permute(1, 0)

    id_dir = os.path.join(config.mmfit_data_dir, w_id)
    np.save(
        os.path.join(id_dir, w_id + "_" + config.sim_acc_file),
        sim_acc.cpu().numpy(),
    )

KeyboardInterrupt: 